In [14]:
"""
Code for accommodating the graph-making need of ECON 3229-2 Money, Banking, and Financial Systems 

University of Missouri-Columbia

Fangda Wang

Fall 2017


https://github.com/vitanova
"""

# I work as teaching assistant for this course, and am told to generate/update lots of figures of macroeconomic
# and monetary time series, mainly from the FRED:
# https://fred.stlouisfed.org/
# although the website has the functionality to plot figures, it's elementary and cannot satisify this course's demand
# and, I am tired of handling excel, since each time I have to build the procedure from the begining
# moreover, there are a number of similar tasks in the foreseeable future, so I decide to write some code 
# to automize the procedure, before it is too late

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

#this is a third-party api for manipulating fred data
from fred import Fred

#first, use the api key to get access to fred, all data in dictionary format
#replace with your own key
fr = Fred(api_key='4bba000775edea91bc01729d217cf3da', response_type='dict')

#now, use the series id to get observations
cpi=fr.series.observations(series_id="HHMSDODNS")
aaa=fr.series.observations(series_id="DPI")
nber=fr.series.observations(series_id="USREC")

#convert the dictionary data to data frame
CPI=pd.DataFrame.from_dict(cpi)
AAA=pd.DataFrame.from_dict(aaa)
NBER=pd.DataFrame.from_dict(nber)

#rename columns
CPI.columns = 'date', 'end', 'start', 'hhm'
AAA.columns = 'date', 'end', 'start', 'dpi'
NBER.columns = 'date', 'end', 'start', 'nber'

#merge the two tables and use date as key
al=pd.merge(CPI, AAA, how='left', on='date')
al=pd.merge(al, NBER, how='left', on='date')
al=al.drop('end_x', axis=1)
al=al.drop('start_x', axis=1)
al=al.drop('end_y', axis=1)
al=al.drop('start_y', axis=1)
al=al.drop('end', axis=1)
al=al.drop('start', axis=1)

al.head(30)

C:\Anaconda3\lib\site-packages\fred\helpers\__init__.py:36: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  frame = frame.convert_objects(convert_numeric=True)
C:\Anaconda3\lib\site-packages\pandas\core\internals.py:3224: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  return self.apply('astype', dtype=dtype, **kwargs)


,date,hhm,dpi,nber
0,1945-10-01,18.71,NaN,1
1,1946-01-01,NaN,NaN,0
2,1946-04-01,NaN,NaN,0
3,1946-07-01,NaN,NaN,0
4,1946-10-01,23.10,NaN,0
5,1947-01-01,NaN,170.989,0
6,1947-04-01,NaN,170.412,0
7,1947-07-01,NaN,178.041,0
8,1947-10-01,28.27,179.846,0
9,1948-01-01,NaN,185.423,0


In [15]:
#to reflect the up-to-date nature of data, we set end date as last available month in fred
#actually, we need data from 1977-07-01, but inflation rate need to be calculated based on last year
#so we get cpi data from 1976
start_date=datetime.date(1977, 4, 1)
end_date=datetime.date(2017, 4, 1)

#since we only plot quarterly data, these monthly observations should be transformed in frequency
index = pd.date_range(start=start_date, end=end_date, freq='3MS')
columns = ['A']
def diff_quarter(d1, d2):
    return (d2.year - d1.year) * 4 + int((d2.month - d1.month)/3)
n_rows=diff_quarter(start_date, end_date) + 1
data = np.array([np.arange(n_rows)]*1).T
df = pd.DataFrame(data, index=index, columns=columns)
df['date']=df.index

#now, use the quarterly format table to extract information from the fred data
new_one=pd.merge(df, al, how='left', on='date')
new_one=new_one.drop('A', axis=1)
new_one.head()

,date,hhm,dpi,nber
0,1977-04-01,556.49,1432.595,0
1,1977-07-01,579.58,1476.478,0
2,1977-10-01,603.00,1526.535,0
3,1978-01-01,627.04,1562.876,0
4,1978-04-01,650.88,1610.855,0


In [16]:
new_one['ratio']=new_one['hhm']/new_one['dpi'] *100
new_one.head()

,date,hhm,dpi,nber,ratio
0,1977-04-01,556.49,1432.595,0,38.844893
1,1977-07-01,579.58,1476.478,0,39.254225
2,1977-10-01,603.00,1526.535,0,39.501223
3,1978-01-01,627.04,1562.876,0,40.120905
4,1978-04-01,650.88,1610.855,0,40.405871


In [17]:
#transform date format from yyyy-mm-dd to yyyy Qq
def tran_date(xx):
    return str(xx.year) + " Q" + str(1+int(xx.month/3))
new_one['new_date']=new_one.apply(lambda row: tran_date(row['date']), axis=1)

#add the zero line, prep two nber series to facilitate making the shaded area between them
new_one['zero']=0
new_one['+nber']=100*new_one['nber']
new_one['-nber']=-new_one['+nber']
new_one.head()

t=new_one
t.head()

,date,hhm,dpi,nber,ratio,new_date,zero,+nber,-nber
0,1977-04-01,556.49,1432.595,0,38.844893,1977 Q2,0,0,0
1,1977-07-01,579.58,1476.478,0,39.254225,1977 Q3,0,0,0
2,1977-10-01,603.00,1526.535,0,39.501223,1977 Q4,0,0,0
3,1978-01-01,627.04,1562.876,0,40.120905,1978 Q1,0,0,0
4,1978-04-01,650.88,1610.855,0,40.405871,1978 Q2,0,0,0


In [18]:
#since fill_between mathod doesnt work well with datetype64, we need to transform it
finally_=t.date.values
fig, ax=plt.subplots(figsize=(16, 9))
ax.plot(finally_, t['ratio'], label='mortgage/income', linewidth=3)

#transform the shaded area into rectangle
import matplotlib.transforms as mtransforms
trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
ax.fill_between(finally_, t['+nber'], t['-nber'], facecolor='gray', interpolate=False, transform=trans, alpha=0.2)

#set the horizontal label as transformed data type
ax.set_xlim([t.date[t.index[0]], t.date[t.index[-1]]])
real_date=[]
trans_date=[]
for i in range(11):
    real_date.append(t.date[t.index[16*i]])
    trans_date.append(t.new_date[t.index[16*i]])
ax.set_xticks(real_date)
ax.set_xticklabels(trans_date, fontsize=14)

ax.set_yticklabels(np.linspace(30, 110, 9), fontsize=14)
ax.grid()
ax.set_ylim(30, 110)
ax.set_ylabel("%", fontsize=16)

ax.legend(loc='best', fontsize=16)
ax.set_title("Volume of Household Mortgage Debt as Percentage of Personal Disposable Income", fontsize=20, position=[.5, 1.03])
plt.savefig('mortgage_income.jpg', dpi=200)